### Introdução

blabla

#### Importações

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import PIL

In [2]:
#Importando modelo
from DNN.models.custom_model import CustomModel
from DNN.layers.layer import Dense
from DNN.layers.activation import ReLU, Sigmoid
from DNN.training.loss import CrossEntropyLoss
from DNN.training.optimizer import GradientDescent
from DNN.training.trainer import Trainer
from DNN.evaluation.evaluate import evaluate
from DNN.evaluation.cross_validation import CrossValidation

#### Lendo datasets

In [3]:
df = pd.read_csv("../datasets/creditcard.csv")

In [4]:
df.groupby("Class")["Class"].count()

Class
0    284315
1       492
Name: Class, dtype: int64

Fazendo a contagem de linhas para cada classe, vemos que há um número bastante desigual de linhas entre as labels 0 e 1, o que pode tornar o treinamento do modelo enviesado. Para atenuar esse problema, utilizarei um método de amostragem chamado Stratified K-Fold, que consiste na separação de amostras que buscam igualar a proporção entre os dois tipos de labels para cada 'fold'.

In [5]:
#Importando Stratified K-Fold do sklearn
from sklearn.model_selection import StratifiedKFold

In [6]:
#Separando entradas e saídas
X = df.loc[:,df.columns != "Class"].copy()
y = df['Class'].copy()

In [7]:
#Realizando a primeira separação entre set de treinamento e de teste com proporção 0.1 de teste.
skf = StratifiedKFold(n_splits=30)

#Obtendo indexadadores das amostras de treino e de teste
train_idx, test_idx = list(skf.split(X,y))[0][0], list(skf.split(X,y))[0][1]

#Aplicando indexadores às entradas e saidas
X_train, y_train = X.loc[X.index[train_idx]].copy(), y[y.index[train_idx]].copy()

Agora, com as amostras de treino e teste, realizarei mais uma separação do set de treino em 9 partes iguais, desse modo, terei 9 folds para validação cruzada e um fold para avaliar o enviesamento do modelo.

In [8]:
#Realizando a separação do set treinamento em 9 folds
skf1 = StratifiedKFold(n_splits=29)

folds = {}

for i, (train_idx, test_idx) in enumerate(skf1.split(X_train, y_train)):
    folds[f"Fold_{i}"] = {'train':{"X": np.array(X_train.loc[X_train.index[train_idx]].copy()), 'y': np.array(y_train[y_train.index[train_idx]].copy())},
                          'test': {"X": np.array(X_train.loc[X_train.index[test_idx]].copy()), 'y': np.array(y_train[y_train.index[test_idx]].copy())}} 

In [9]:
#Inicializando modelo, loss_func e otimizador 
model = CustomModel([
    Dense(len(X.columns), 300),
    ReLU(),
    Dense(300,1),
    Sigmoid()])

loss_func = CrossEntropyLoss()
optimizer = GradientDescent(learning_rate=0.01)

In [10]:
trainer = Trainer(model, loss_func, optimizer)
#trainer.train(X_train.T, y_train, epochs=2500)

In [11]:
cross_val = CrossValidation(model, trainer)

In [12]:
cross_val.validate(folds, epochs = 10)

D:\OneDrive - unb.br\Área de Trabalho\Materiais para estudo\CIS_IEEE\Trainee\4_Redes_Neurais\notebooks\DNN\layers\activation.py:14: RuntimeWarning: overflow encountered in exp
  self.A = (1 / (1 + np.exp(-X)))


Epoch 1/10, Loss: 0.035784


C:\Users\thiag\anaconda3\envs\cis_ieee-env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\OneDrive - unb.br\Área de Trabalho\Materiais para estudo\CIS_IEEE\Trainee\4_Redes_Neurais\notebooks\DNN\layers\activation.py:14: RuntimeWarning: overflow encountered in exp
  self.A = (1 / (1 + np.exp(-X)))


Epoch 1/10, Loss: 0.035784


C:\Users\thiag\anaconda3\envs\cis_ieee-env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\OneDrive - unb.br\Área de Trabalho\Materiais para estudo\CIS_IEEE\Trainee\4_Redes_Neurais\notebooks\DNN\layers\activation.py:14: RuntimeWarning: overflow encountered in exp
  self.A = (1 / (1 + np.exp(-X)))


Epoch 1/10, Loss: 0.035784


KeyboardInterrupt: 